In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/Madison/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
# Upon review of the exoplanet archive website the following variables appear to have impact
# on if a exoplanet is confirmed. koi_time0bk was removed as it is a time it 
# was observed not a feature of the object. 
# Error was added back in and accuracy increase by ~0.05.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]

In [5]:
# removing variables with low variance (removing insignificant variables)
from sklearn.feature_selection import VarianceThreshold
# V2 80% model was more accurate
# V3 70% model became less accurate
# V4 75%
var_model = VarianceThreshold(threshold=(.80 * (1 - .80)))
selected_data = var_model.fit_transform(selected_features)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
y = df["koi_disposition"]
y_binary_encoded = pd.get_dummies(y)

In [7]:
# setting y to be a single output as it is either confirmed or anything else and it has to be
# confirmed to be on the Exoplanet Archive Confirmed Planet table
y_confirmed = y_binary_encoded["CONFIRMED"]

In [8]:
from sklearn.model_selection import train_test_split

#spliting training data, setting back to x and y for ease of modeling
X_train, X_test, y_train, y_test = train_test_split(selected_data, y_confirmed, random_state=57)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
# only X variable needs to be scaled since y is all binary encoding
# using training data to create the scaler 
X_MinMaxScaler = MinMaxScaler().fit(X_train)

In [10]:
X_train_scaled = X_MinMaxScaler.transform(X_train)
X_test_scaled = X_MinMaxScaler.transform(X_test)

# Train the Model



In [11]:
# first model tested is linear regression to get a base comparison 
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression()
model2.fit(X_train_scaled, y_train)

/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.8380698073621972
Testing Data Score: 0.8215102974828375


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
# oringally ran C with 1, 5, 10 and 10 was the best so added C options
# next ran with 1, 5, 10, 20, 50 and 50 was the best so tried another set (see final set below)
param_grid2 = {'C': [10, 50, 100, 1000, 5000, 10000],
              'fit_intercept': [True, False],
              'max_iter': [100, 200, 300]}
grid2 = GridSearchCV(model2, param_grid2, verbose=3)

In [14]:
# Train the model with GridSearch
grid2.fit(X_train_scaled, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.846, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.870, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.872, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

[CV]  C=10, fit_intercept=True, max_iter=100, score=0.866, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.846, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=200 ..........................


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.879, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................
[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.888, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.880, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................
[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.880, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................
[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.878, total=   0.2s


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:   19.9s finished
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver op

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [10, 50, 100, 1000, 5000, 10000],
                         'fit_intercept': [True, False],
                         'max_iter': [100, 200, 300]},
             verbose=3)

In [15]:
print(grid2.best_params_)
print(grid2.best_score_)
# verison 1 of model 2 with all error added in and no low variance columns removed best score
# 0.8806029370028889
# verison 2 of model 2 with all error added in and 80% on low variance best score
# 0.8819369956119605
# verison 3 of model 2 with all error added in and 70% on low variance best score
# 0.8655338781391221
# verison 4 of model 2 with all error added in and 75% on low variance best score
# 0.8632459849074727
# verison 5 of model 2 with all error added in, date + error, and 80% on low variance best score
# 0.8806012996747175
# verison 6 of model 2 with all error added in and 80% on low variance best score
# 0.8813659319308101
# best is version 2 80% for the low variance

{'C': 5000, 'fit_intercept': False, 'max_iter': 300}
0.8819369956119605


# Save the Model

In [16]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'mbfisher_model2.sav'
joblib.dump(grid2, filename)

['mbfisher_model2.sav']

In [17]:
predict_values = grid2.predict(X_test_scaled)
pd.DataFrame({"Prediction of Status": predict_values, "Actual Status": y_test})

Prediction of Status  Actual Status
1744                     1              0
4323                     0              0
2919                     0              1
3391                     0              1
3691                     0              0
...                    ...            ...
5459                     0              0
4007                     0              0
1761                     0              0
2068                     0              0
2152                     0              1

[1748 rows x 2 columns]